In [ ]:

    
import os
import numpy as np
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Conv2D , BatchNormalization , Activation , MaxPool2D , Input , Dropout , ZeroPadding2D , Conv2DTranspose , Concatenate
from tensorflow.keras.applications import DenseNet201

from tensorflow.keras.models import Model

from tensorflow.keras.applications import InceptionResNetV2
def conv_block(inputs , num_filters):
  x = Conv2D(num_filters , 3 , padding= "same" )(inputs)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)
  x = Dropout(0.2)(x)

  x = Conv2D(num_filters , 3 , padding= "same" )(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)
  x = Dropout(0.2)(x)
  x = squeeze_excite_block(x)
  return x



def decoder_block(inputs , skip_features , num_filters):

  x = Conv2DTranspose(num_filters , (2, 2) , strides = 2 , padding = "same")(inputs)
  x = Concatenate()([x , skip_features ])
  x = conv_block(x , num_filters)

  return x


In [ ]:

# Even the skip features were extracted for the Decoder purposes to feed in the required Skip Features

def build_DenseNet210_Unet(input_shape):

  ##Input
  inputs = Input(input_shape)

  ## Pre-Trained Encoder
  encoder_feature = []
    
  encoder = DenseNet201(include_top = False , weights = "imagenet" , input_tensor = inputs)

  # Skip Connections which are the Features and we will access these Features and Feed it in the Decoder
  #First Feature is the input shaped image
  s1 = encoder.get_layer("input_1").output                         # 256 * 256

  s2 = encoder.get_layer("conv1/relu").output                       # 128 * 128 

  s3 = encoder.get_layer("pool2_relu").output       # 64 * 64 

  s4 = encoder.get_layer("pool3_relu").output       # 32 * 32
  
  for feature in [s1 , s2 , s3 , s4]:
        encoder_feature.append(feature)
  ## BottleNeck or bridge 
  b1 = encoder.get_layer("pool4_relu").output       # 16 * 16

  ## Decoder
  d1 = decoder_block(b1 , s4 , 512)                                # 32 * 32 * 512 where 512 is the number of Features we extracted 

  d2 = decoder_block(d1 , s3 , 256)                                # 64 * 64

  d3 = decoder_block(d2 , s2 , 128)                                # 128 * 128

  d4 = decoder_block(d3 , s1 , 64)                                # 128 * 128

  ## OUTPUT 

  outputs = Conv2D( 1 ,  1  ,  padding = "same" ,  activation = "sigmoid")(d4)

  
  return outputs , d4 , encoder_feature
  #encoder.summary()

    



In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
from keras.models import *
from keras.layers import Input, Conv2D, UpSampling2D, BatchNormalization, Activation, add,average,concatenate
from keras.layers.core import Lambda
from keras.optimizers import *
from keras.losses import binary_crossentropy

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, Multiply, AveragePooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import InceptionResNetV2

from glob import glob
from sklearn.model_selection import train_test_split
import cv2
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Conv2D , BatchNormalization , Activation , MaxPool2D , Input , Dropout , ZeroPadding2D , Conv2DTranspose , Concatenate


def squeeze_excite_block(inputs, ratio=8):
    init = inputs       ## (b, 128, 128, 32)
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)     ## (b, 32)   -> (b, 1, 1, 32)
    se = Reshape(se_shape)(se)
    se = Dense(filters//ratio, activation="relu", use_bias=False)(se)
    se = Dense(filters, activation="sigmoid", use_bias=False)(se)

    x = Multiply()([inputs, se])
    return x

def ASPP(x, filter):
    shape = x.shape

    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(x)
    y1 = Conv2D(filter, 1, padding="same")(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = Dropout(0.2)(y1)

    y1 = UpSampling2D((shape[1], shape[2]), interpolation="bilinear")(y1)
    
    y2 = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(x)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)
    y2 = Dropout(0.2)(y2)
    
    y3 = Conv2D(filter, 3, dilation_rate=6, padding="same", use_bias=False)(x)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)
    y3 = Dropout(0.2)(y3)
    
    y4 = Conv2D(filter, 3, dilation_rate=12, padding="same", use_bias=False)(x)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)
    y4 = Dropout(0.2)(y4)
    
    y5 = Conv2D(filter, 3, dilation_rate=18, padding="same", use_bias=False)(x)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)
    y5 = Dropout(0.2)(y5)
    
    y = Concatenate()([y1, y2, y3, y4, y5])

    y = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)
    y= Dropout(0.2)(y)
    
    return y


def encoder1(inputs):
    skip_connections = []
    model = InceptionResNetV2(include_top = False  , weights = "imagenet" , input_tensor = inputs)
    #Skip Connection
    s1 = model.get_layer("input_1").output                 #Shape will be 256 * 256

    s2 = model.get_layer("activation_2").output         # 125 * 125 
    s2 = ZeroPadding2D( ((2 , 1) ,  ( 2 , 1)) )(s2)          # 128 * 128


    s3 = model.get_layer("activation_4").output        # 60 * 60
    s3 = ZeroPadding2D( ((2 , 2 ) , (2, 2)) )(s3)         # 64 * 64 


    s4 = model.get_layer("activation_74").output                              # 29*29
    s4 = ZeroPadding2D( ((2 , 1) ,  ( 2 , 1)) )(s4) 
                                                                                # 32 * 32
    # Padded with 2 on top , 1 in bottom     2 in left and 1 in right


    b1 = model.get_layer("activation_161").output                      # 14 * 14

    #model = VGG19(include_top=False, weights="imagenet", input_tensor=inputs)
    names = ["input_1", "activation_2", "activation_4", "activation_74"]

    for feature in [s1 , s2 , s3 , s4]:

        skip_connections.append(feature)


    output = model.get_layer("activation_161").output
    output = ZeroPadding2D( ((1 , 1) , (1, 1)) )(output)                         # 16 * 16 

    return output, skip_connections

def decoder1(inputs, skip_connections):
    num_filters = [512,256, 128, 64]
    skip_connections.reverse()

    x = inputs
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation="bilinear")(x)
        x = Concatenate()([x, skip_connections[i]])
        x = conv_block(x, f)

    return x

def output_block(inputs):
    x = Conv2D(1, 1, padding="same")(inputs)
    x = Activation("sigmoid")(x)
    return x

def encoder2(inputs):
    num_filters = [ 64, 128, 256 , 512]
    skip_connections = []

    x = inputs

    for i, f in enumerate(num_filters):
        x = conv_block(x, f)
        skip_connections.append(x)
        x = MaxPool2D((2, 2))(x)

    return x, skip_connections

def decoder2(inputs, skip_1, skip_2 ,encoder2_skip_connection ):
    num_filters = [512 , 256, 128, 64]
    skip_2.reverse()
    encoder2_skip_connection.reverse()
    x = inputs
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation="bilinear")(x)
        x = Concatenate()([x, skip_1[i], skip_2[i] , encoder2_skip_connection[i+1]])
        #x = Concatenate()([x, skip_1[i], skip_2[i]])
        x = conv_block(x, f)

    return x

h_heuns_method=0.5

def res_block(x, nb_filters, strides):
    res_path = BatchNormalization()(x)
    res_path = Activation(activation='relu')(res_path)
    res_path = Conv2D(filters=nb_filters[0], kernel_size=(3, 3), padding='same' ,strides=strides[0])(res_path)
    res_path = Dropout(0.2)(res_path)

    res_path = BatchNormalization()(res_path)
    res_path = Activation(activation='relu')(res_path)
    res_path = Conv2D(filters=nb_filters[1], kernel_size=(3, 3), padding='same',strides=strides[1])(res_path)
    
    res_path = Dropout(0.2)(res_path)
    
    hpath = Lambda(lambda x: x * h_heuns_method)(res_path)

    shortcut = Conv2D(nb_filters[1], kernel_size=(1, 1),strides=strides[0])(x)
    shortcut = BatchNormalization()(shortcut)
    shortcut = Activation(activation='relu')(shortcut)
    shortcut = Dropout(0.2)(shortcut)
    
    res_path = add([shortcut, hpath])#suma corta
    return res_path

def res_block2(x,y,nb_filters, strides):
    res_path = BatchNormalization()(x)
    res_path = Activation(activation='relu')(res_path)
    res_path = Conv2D(filters=nb_filters[0], kernel_size=(3, 3), padding='same' ,strides=strides[0])(res_path)
    res_path = Dropout(0.2)(res_path)

    res_path = BatchNormalization()(res_path)
    res_path = Activation(activation='relu')(res_path)
    res_path = Conv2D(filters=nb_filters[1], kernel_size=(3, 3), padding='same',strides=strides[1])(res_path)
    
    res_path = Dropout(0.2)(res_path)

    hpath = Lambda(lambda x: x * h_heuns_method)(res_path)

    shortcut = Conv2D(nb_filters[1], kernel_size=(1, 1),strides=strides[0])(x)
    shortcut = BatchNormalization()(shortcut)
    #shortcut = Activation(activation='relu')(shortcut)
    shortcut = Dropout(0.2)(shortcut)
    res_path = add([shortcut, hpath])

    res_path = average([y, res_path])
    return res_path

def encoder(x):
    to_decoder = []
    main_path = Conv2D(filters=32, kernel_size=(3, 3), padding='same', strides=(1, 1))(x)  ## 512
    main_path = BatchNormalization()(main_path)
    main_path = Activation(activation='relu')(main_path)
    main_path = Dropout(0.2)(main_path)

    main_path = Conv2D(filters=32 , kernel_size=(3, 3), padding='same', strides=(1, 1))(main_path)## 512
    main_path = BatchNormalization()(main_path)
    main_path = Activation(activation='relu')(main_path)
    main_path = Dropout(0.2)(main_path)
    
    hpath = Lambda(lambda x: x * h_heuns_method)(main_path)

    shortcut = Conv2D(filters=32, kernel_size=(1, 1), strides=(1, 1))(x)                            ## 512
    shortcut = BatchNormalization()(shortcut)
    #shortcut = Activation(activation='relu')(shortcut)
    shortcut = Dropout(0.2)(shortcut)

    main_path = add([shortcut, hpath])    ## 512 is shortcut and 512 is hpath

    to_decoder.append(main_path)


    s1 = Conv2D(filters=64, kernel_size=(1, 1), strides=(2, 2))(x)#We can change Kernel Size                                 ## 512 to 256
    s1 = BatchNormalization()(s1)
    #s1 = Activation(activation='relu')(s1)
    s1 = Dropout(0.2)(s1)
    
    main_path = res_block2(main_path,s1, [64, 64], [(2, 2), (1, 1)])                         # mainPath becames 512 to 256    to fit s1 which is 256
    to_decoder.append(main_path)


    s2 = Conv2D(filters=128, kernel_size=(1, 1), strides=(2, 2))(to_decoder[1])              # 256 to 128
    s2 = BatchNormalization()(s2)
    #s2 = Activation(activation='relu')(s2)
    s2 = Dropout(0.2)(s2)
    
    main_path = res_block2(main_path,s2, [128, 128], [(2, 2), (1, 1)])                    # main_path became from 256 to 128 
    to_decoder.append(main_path)
    
    s3 = Conv2D(filters=256, kernel_size=(1, 1), strides=(2, 2))(to_decoder[2])
    s3 = BatchNormalization()(s3)
    #s3 = Activation(activation='relu')(s3)
    s3 = Dropout(0.2)(s3)
    
    main_path = res_block2(main_path,s3, [256, 256], [(2, 2), (1, 1)])                     # 128 to 64
    to_decoder.append(main_path)

    s4 = Conv2D(filters=512, kernel_size=(1, 1), strides=(2, 2))(to_decoder[3])        
    s4 = BatchNormalization()(s4)
    #s4 = Activation(activation='relu')(s4)
    s4 = Dropout(0.2)(s4)
    
    
    main_path = res_block2(main_path,s4, [512, 512], [(2, 2), (1, 1)])
    to_decoder.append(main_path)

    return to_decoder

def decoder(x, from_encoder):
    main_path = UpSampling2D(size=(2, 2) , interpolation="bilinear")(x)# 32 * 32
    main_path1 = concatenate([main_path, from_encoder[-1]], axis=3)
    main_path = res_block(main_path1, [512, 512], [(1, 1), (1, 1)])

    main_path = UpSampling2D(size=(2, 2), interpolation="bilinear")(main_path)###64x64
    main_path = concatenate([main_path, from_encoder[3]], axis=3)#
    u1 = UpSampling2D(size=(2, 2))(main_path1)#
    u1 = Conv2D(256, kernel_size=(1, 1),strides=(1, 1))(u1)
    u1 = BatchNormalization()(u1)   
    #u1 = Activation(activation='relu')(u1)
    u1 = Dropout(0.2)(u1)
    
    
    main_path = res_block2(main_path,u1, [256, 256], [(1, 1), (1, 1)])#

    main_path = UpSampling2D(size=(2, 2) , interpolation="bilinear")(main_path)#128x128
    main_path2 = concatenate([main_path, from_encoder[2]], axis=3)#
    main_path = res_block(main_path2, [128, 128], [(1, 1), (1, 1)])#

    main_path = UpSampling2D(size=(2, 2) , interpolation="bilinear" )(main_path)#256x256
    main_path = concatenate([main_path, from_encoder[1]], axis=3)#256x256

    u2 = UpSampling2D(size=(2,2) , interpolation="bilinear" )(main_path2)#
    u2 = Conv2D(64, kernel_size=(1, 1),strides=(1, 1))(u2)#
    u2 = BatchNormalization()(u2)
    #u2 = Activation(activation='relu')(u2)
    u2 = Dropout(0.2)(u2)
    main_path3 = res_block2(main_path,u2, [64, 64], [(1, 1), (1, 1)])#256x256

    
    main_path = UpSampling2D(size=(2, 2) , interpolation="bilinear" )(main_path3)#512 * 512
    main_path = concatenate([main_path, from_encoder[0]], axis=3)# 512 * 512
    
    #u3 = UpSampling2D(size=(2,2) , interpolation="bilinear")(main_path3)#
    #u3 = Conv2D(64, kernel_size=(1, 1),strides=(1, 1))(u3)#
    #u3 = BatchNormalization()(u3)
    #u3 = Activation(activation='relu')(u3)
    #u3 = Dropout(0.2)(u3)
    
    main_path = res_block(main_path, [ 64, 64], [(1, 1), (1, 1)])#


    return main_path
  




def build_model(input_shape):
    inputs = Input(input_shape)
    
    output1 , features_map_Decoder1 , feature_map_Encoder1 = build_DenseNet210_Unet(inputs)
    skip_1 = feature_map_Encoder1
    #x, skip_1 = encoder1(inputs)
    #x = ASPP(x, 64)
    #x = decoder1(x, skip_1)
    #output1 = output_block(x)

    x1 = inputs * output1

    #x = concatenate([x1 , features_map_Decoder1] , axis = 3)
    # 2ND Encoder is taking Ouput of first Unet 
    to_decoder = encoder(x1)
    path = res_block(to_decoder[-1], [1024, 1024], [(2, 2), (1, 1)])####bridge
    #x = ASPP(x, 64)
    path = decoder(path, from_encoder=to_decoder)
    output2 = output_block(path)
    x2 = inputs * output2
    

    x = concatenate([x1 , x2 ] , axis = 3)
    x, skip_2 = encoder2(x)
    x = ASPP(x, 64)
    
    x = decoder2(x, skip_1, skip_2 , to_decoder) ## Now Last Decoder is taking all the Features from Encoder 1 , Encoder 2 and Encoder 3 or say from Bridge 3 . Why it is not taking the outputs from Unet1 and Unet2
    output3 = output_block(x)
    #o1 = []
    #o2 = []
    #o3 = []
    #o1.append(output1)
    #o2.append(output2)
    #o3.append(output3)
    
    
    #outputs = concatenate([output1, output2 , output3, inputs] , axis=3)
    
    #output = output_block(outputs)
    
    model = Model(inputs, output3)
    #return output , model
    return model


In [ ]:
def load_data(paths , split = 0.15):
  images = []
  masks = []
  numImages = 0
  numMasks = 0

  for path in paths:
    
        
    images.extend(sorted(glob(os.path.join(path , "images/*"))))
    masks.extend(sorted(glob(os.path.join(path , "label/*"))))
    if "training" in path:
      for i in ["00", "01" , "02" , "03" , "04" , "05" , "07", "08" , "09" , "10" , "11" , "12" , "13" , "14" , "16" , "17" , "18" , "19" , "21" , "22" , "24" , "26" , "27" ]:
        images.remove("../input/chagas/ChagasTraining/training-20210525T143718Z-001/training/images/i8{}.xml".format(i))

      n = "training"
    if "Test" in path:
      n= "Test"
    if "Val" in path:

      n= "Validation"

    print("{}PathImagesAre:".format(n),len(images) - numImages)
    print("{}PathImagesAre:".format(n),len(masks) - numMasks)

    numImages = len(images)
    numMasks = len(masks)

  total_size = len(images)
  valid_size = int(split * total_size)
  test_size = int(split * total_size)
  print("Number of images in Total , Validation images and Test Images : " , total_size , valid_size ,test_size  )

  train_x , test_x = train_test_split(images , test_size = test_size , random_state = 42 )
  train_y , test_y = train_test_split(masks , test_size = test_size , random_state = 42 )

  train_x , valid_x = train_test_split(train_x , test_size = valid_size , random_state = 42 )
  train_y , valid_y = train_test_split(train_y , test_size = valid_size , random_state = 42 )

  return (train_x , train_y) , (valid_x , valid_y ) , (test_x , test_y) 

def read_image(path):
  path = path.decode()
  x = cv2.imread(path , cv2.IMREAD_COLOR)
  #x = cv2.resize(x , (256 , 256))
  x = x/255.0
  #Size is 256 * 256 * 3

  return x

def read_mask(path):
  path = path.decode()
  x = cv2.imread(path , cv2.IMREAD_GRAYSCALE)
  #x = cv2.resize(x , (256 , 256))
  x = x/255.0
  
  #Size is 256 * 256
  x = np.expand_dims(x , axis = -1)
  #Size becames 256 * 256 * 1

  return x

def tf_parse(imagepath , maskpath):
  def _parse(imagepath , maskpath):
    x = read_image(imagepath)
    y = read_mask(maskpath)

    return x , y

  x , y = tf.numpy_function(_parse , [imagepath , maskpath] , [tf.float64 , tf.float64] )
  x.set_shape([512 , 512 , 3])
  y.set_shape([512, 512, 1])

  return x , y 


def tf_dataset( imagepath , maskpath , batch = 2):
  dataset = tf.data.Dataset.from_tensor_slices((imagepath , maskpath))
  dataset = dataset.map(tf_parse)
  dataset = dataset.batch(batch)
  dataset = dataset.repeat()
  return dataset

if __name__ == "__main__":
  paths = ["../input/chagas/ChagasTraining/training-20210525T143718Z-001/training" , "../input/chagas/ChagasTest/Test", "../input/chagas/ChagasValidation/Validation"] 
  (train_x , train_y) , (valid_x , valid_y ) , (test_x , test_y)  =   load_data(paths)

  print("Number of images in Train , Validation images and Test Images : " ,len(train_x) , len(valid_x ) , len(test_x) )

  ds = tf_dataset(test_x , test_y)
  for x, y in ds:
    
    print(x.shape , y.shape)
    break
    


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model





In [ ]:
# Train 
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from tensorflow.keras.metrics import Recall , Precision 

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)


In [ ]:
pip install segmentation_models

In [ ]:
%env SM_FRAMEWORK=tf.keras

In [ ]:
if __name__ == "__main__":
    from segmentation_models.losses import bce_jaccard_loss
    from segmentation_models.metrics import iou_score
    np.random.seed(42)
    tf.random.set_seed(42)
    #HYPERPARAMETER


    batch = 2
    lr = 8.00E-05
    epochs = 2
    opt = tf.keras.optimizers.Adam(lr)
    metrics = ["acc" , Recall() , Precision() , iou]
    #Data
    train_dataset = tf_dataset(train_x, train_y, batch=batch)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)
    model = build_model((512 , 512 , 3))
    #final_output , model = build_model((512 , 512 , 3))
    #model.summary()
    #model.compile(loss=bce_jaccard_loss, optimizer=opt, metrics=metrics)
    model.compile(optimizer= opt, loss=bce_jaccard_loss, metrics=[dice_loss,iou_coeff,precision,recall , ACL5])

    callbacks = [
        ModelCheckpoint("files/model.h5"),
        ReduceLROnPlateau(monitor='iou', factor=0.1, patience=4),
        #CSVLogger("/content/drive/MyDrive/data.csv"),
        TensorBoard(),
        EarlyStopping(monitor='iou', patience=15, restore_best_weights=False)
    ]
    #Number Of Batches
    train_steps = len(train_x)//batch
    valid_steps = len(valid_x)//batch

    if len(train_x) % batch != 0:
        train_steps += 1
    if len(valid_x) % batch != 0:
        valid_steps += 1

    model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=epochs,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks,
        shuffle = False)

In [ ]:
    model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=30,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks,
        shuffle = False)


In [ ]:
import os
  
# Directory
directory = "RESULT_DenseNet210WithDropout"
  
# Parent Directory path
parent_dir = "./"
  
# Path
path = os.path.join(parent_dir, directory)
  
# Create the directory
# 'GeeksForGeeks' in
# '/home / User / Documents'
os.mkdir(path)

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm
#from data import load_data, tf_dataset
#from train import iou

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = np.expand_dims(x, axis=-1)
    return x

def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

if __name__ == "__main__":
    ## Dataset
    
    batch_size = 8
    #(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data("/content/drive/MyDrive/Test")

    test_dataset = tf_dataset(test_x, test_y, batch=batch_size)

    test_steps = (len(test_x)//batch_size)
    if len(test_x) % batch_size != 0:
        test_steps += 1

    #with CustomObjectScope({'iou': iou}):
    #    model = tf.keras.models.load_model("/content/drive/model.h5")

    #model.evaluate(test_dataset, steps=test_steps)

    for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
        x = read_image(x)
        y = read_mask(y)
        y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.5
        h, w, _ = x.shape
        white_line = np.ones((h, 10, 3)) * 255.0

        all_images = [
            x * 255.0, white_line,
            mask_parse(y), white_line,
            mask_parse(y_pred) * 255.0
        ]
        image = np.concatenate(all_images, axis=1)
        
        cv2.imwrite(f"./RESULT_DenseNet210WithDropout/{i}.png", image)